## Test 'creation of outer shell mesh'

In [1]:
import os
import napari
import numpy as np
from scipy.spatial import Delaunay
from scipy.interpolate import LinearNDInterpolator, NearestNDInterpolator
import trimesh
import math
import open3d as o3d
from trimesh.proximity import signed_distance
# from MeshPrep import create_outer_shell_mesh
from trimesh.creation import icosphere
from typing import List

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def create_sample_spheres(
        n: int,
        radiuses: List[float]
) -> List[trimesh.Trimesh]:
    """
    Create n spherical meshes displaced one next to the other and with centers aligned.

    Parameters:
    -----------
    n: (int)
        The number of spheres to create
    radiuses: (List[int])
        The radius of the spheres

    Returns:
    --------
    spheres: (List[trimesh.Trimesh])
        A list of meshes of spheres
    """
    spheres = []
    displacement = np.array([0.0, 0.0, 0.0])
    for i in range(n):
        curr_sphere = icosphere(radius=radiuses[i])
        curr_sphere.vertices += displacement
        displacement[0] += 2 * radiuses[i]
        spheres.append(curr_sphere)

    return spheres

#### Example with spheres

In [ ]:
spheres = create_sample_spheres(3, [1, 1, 1])

In [ ]:
neighbors_lst = [[1], [0, 2], [1]]

In [ ]:
# Calculate the length of each edge
edges = spheres[0].edges
edge_lengths = spheres[0].vertices[edges[:, 0]] - spheres[0].vertices[edges[:, 1]]
edge_lengths = trimesh.util.row_norm(edge_lengths)

# Get the shortest edge length
shortest_edge_length = min(edge_lengths)
print(shortest_edge_length)

0.13828317354716763


In [ ]:
outer_shell_mesh, outer_shell_cloud, outer_shell_pts = create_outer_shell_mesh(
    spheres,
    neighbors_lst,
    shortest_edge_length / 3,
    shortest_edge_length / 3
)

---------------------------------------------------
Current mesh: <trimesh.Trimesh(vertices.shape=(642, 3), faces.shape=(1280, 3))>, curr neighbors: [1]
Original num of vertices: (642, 3)
Max distance: 2.0, mean: 1.168977044593609, std: 0.5526613728851824
Remaining num of vertices: (635, 3)
---------------------------------------------------
Current mesh: <trimesh.Trimesh(vertices.shape=(642, 3), faces.shape=(1280, 3))>, curr neighbors: [0, 2]
Original num of vertices: (642, 3)
Max distance: 2.0, mean: 1.168977044593609, std: 0.5526613728851824
Remaining num of vertices: (635, 3)
Max distance: 1.98105595120421, mean: 1.15987007327802, std: 0.5488123902392724
Remaining num of vertices: (628, 3)
---------------------------------------------------
Current mesh: <trimesh.Trimesh(vertices.shape=(642, 3), faces.shape=(1280, 3))>, curr neighbors: [1]
Original num of vertices: (642, 3)
Max distance: 2.0, mean: 1.168977044593609, std: 0.5526613728851824
Remaining num of vertices: (635, 3)


In [ ]:
viewer = napari.Viewer()
for sphere in spheres:
    viewer.add_surface((sphere.vertices, sphere.faces))
viewer.add_surface((outer_shell_mesh.vertices, outer_shell_mesh.faces), opacity=0.5, blending="additive")
viewer.add_points(outer_shell_pts, face_color="red", size=0.1)

<Points layer 'outer_shell_pts' at 0x1bb4ebe2e60>

In [ ]:
o3d.io.write_point_cloud(r"../../../shell_pcd_2.ply", outer_shell_cloud)

True

#### Example with cell meshes

In [ ]:
mesh1 = trimesh.load_mesh(r"N:\Users\Federico_Carrara\Meshes_for_Simulation\examples\cell_clump_intestine\cell_clumps\clean_clump_16_cells\clean_meshes\cell_210.stl")
mesh2 = trimesh.load_mesh(r"N:\Users\Federico_Carrara\Meshes_for_Simulation\examples\cell_clump_intestine\cell_clumps\clean_clump_16_cells\clean_meshes\cell_228.stl")

In [ ]:
mesh_lst = [mesh1, mesh2]
neighbors_lst = [[1], [0]]

# Calculate the length of each edge
edges = mesh1.edges
edge_lengths = mesh1.vertices[edges[:, 0]] - mesh1.vertices[edges[:, 1]]
edge_lengths = trimesh.util.row_norm(edge_lengths)

# Get the shortest edge length
shortest_edge_length = min(edge_lengths)
print(shortest_edge_length)

0.09302981215157372


In [ ]:
outer_shell_mesh, outer_shell_cloud, outer_shell_pts = create_outer_shell_mesh(
    mesh_lst,
    neighbors_lst,
    shortest_edge_length / 3,
    shortest_edge_length / 3
)

---------------------------------------------------
Current mesh: <trimesh.Trimesh(vertices.shape=(17361, 3), faces.shape=(34722, 3))>, curr neighbors: [1]
Original num of vertices: (17361, 3)


### Test new OOP approach

Get outer shell from single meshes

In [3]:
from OuterShell import ExtendedTrimesh, OuterShell

In [5]:
### SPHERES ###

# Initialize synthtic input
spheres = create_sample_spheres(3, [1, 1, 1])
neighbors_lst = [[1], [0, 2], [1]]

# Calculate the length of each edge
edges = spheres[0].edges
edge_lengths = spheres[0].vertices[edges[:, 0]] - spheres[0].vertices[edges[:, 1]]
edge_lengths = trimesh.util.row_norm(edge_lengths)

# Get the shortest edge length
shortest_edge_length = min(edge_lengths)

# Transform in ExtendedTrimesh format
extended_meshes = []
for i, sphere in enumerate(spheres):
    extended_meshes.append(ExtendedTrimesh(neighbors=neighbors_lst[i], vertices=sphere.vertices, faces=sphere.faces))

In [7]:
### CELL MESHES ###
root_dir = r"N:\Users\Federico_Carrara\Meshes_for_Simulation\examples\cell_clump_intestine\cell_clumps\clean_clump_16_cells\clean_meshes"
mesh1 = trimesh.load_mesh(os.path.join(root_dir, "cell_193.stl"))
mesh2 = trimesh.load_mesh(os.path.join(root_dir, "cell_228.stl"))
mesh3 = trimesh.load_mesh(os.path.join(root_dir, "cell_171.stl"))

mesh_lst = [mesh1, mesh2, mesh3]
neighbors_lst = [[1, 2], [0, 2], [0, 1]]

# Calculate the length of each edge
edges = mesh1.edges
edge_lengths = mesh1.vertices[edges[:, 0]] - mesh1.vertices[edges[:, 1]]
edge_lengths = trimesh.util.row_norm(edge_lengths)

# Get the shortest edge length
shortest_edge_length = min(edge_lengths)
print(shortest_edge_length)

# Transform in ExtendedTrimesh format
extended_meshes = []
for i, mesh in enumerate(mesh_lst):
    extended_meshes.append(ExtendedTrimesh(neighbors=neighbors_lst[i], vertices=mesh.vertices, faces=mesh.faces))

0.08826070100385765


In [8]:
outer_shell = OuterShell(
    meshes=extended_meshes, 
    neighbors_lst=neighbors_lst, 
    min_edge_length=shortest_edge_length
)

In [9]:
outer_shell.get_shell_point_cloud(dist_threshold=10)

In [10]:
viewer = napari.Viewer()
viewer.add_points(outer_shell.points, size=0.1)

<Points layer 'Points' at 0x2085ecde7d0>

In [12]:
# # Check that k-closest points are correctly extracted
# k_closest_idxs_1 = np.concatenate([
#     outer_shell._meshes[0].k_closest_dict[1],
#     outer_shell._meshes[0].k_closest_dict[2],
# ])
# k_closest_idxs_2 = np.concatenate([
#     outer_shell._meshes[1].k_closest_dict[0],
#     outer_shell._meshes[1].k_closest_dict[2],
# ])
# k_closest_idxs_3 = np.concatenate([
#     outer_shell._meshes[2].k_closest_dict[0],
#     outer_shell._meshes[2].k_closest_dict[1],
# ])

# k_closest_pts_1 = outer_shell._meshes[0].points[k_closest_idxs_1]
# k_closest_pts_2 = outer_shell._meshes[1].points[k_closest_idxs_2]
# k_closest_pts_3 = outer_shell._meshes[2].points[k_closest_idxs_3]

# viewer = napari.Viewer()
# viewer.add_points(outer_shell.points, size=0.1)
# viewer.add_points(k_closest_pts_1, size=0.1, face_color="green")
# viewer.add_points(k_closest_pts_2, size=0.1, face_color="red")
# viewer.add_points(k_closest_pts_3, size=0.1, face_color="orange")

<Points layer 'k_closest_pts_3' at 0x20873592aa0>

#### Interpolation trials

##### 1. Fit a global model, evaluate it on the gaps
Pros:
- Global model allows to capture features common to all the point cloud and extend them to the gaps

Cons:
- Parameters may depend on the particular sample/shape, and hence it would be difficult to generalize.
- It is not trivial to individuate gaps and to place appropriate sampling grids. 

In [ ]:
# Train the chosen model on existing data
x, y, z = outer_shell.points[:, 0], outer_shell.points[:, 1], outer_shell.points[:, 2]
model = NearestNDInterpolator(np.column_stack([x, y]), z)

### Get grid of x, y values at the gaps
# 0. Compute the step of the grid as the average distance between points over all the meshes
grid_step = np.mean([mesh.mean_point_distance for mesh in outer_shell._meshes]) / math.sqrt(2)

# 1. Get all pairs of neighbors
neighbor_pairs = set()
for idx, neighbors in enumerate(outer_shell._neighbors_lst):
    for neighbor in neighbors:
        pair = tuple(sorted((idx, neighbor)))
        neighbor_pairs.add(pair)

# 2. For each pair:
new_shell_points = []
all_closest_points = []
pred_grids = []
for idx_1, idx_2 in neighbor_pairs:
    # 2.a. Get closest points for each cell in the pair
    mesh_1, mesh_2 = outer_shell._meshes[idx_1], outer_shell._meshes[idx_2]
    closest_point_idxs_1 = mesh_1.k_closest_dict[idx_2]
    closest_point_idxs_2 = mesh_2.k_closest_dict[idx_1]
    closest_points = np.concatenate(
        [mesh_1.points[closest_point_idxs_1], mesh_2.points[closest_point_idxs_2]]
    )
    all_closest_points.append(closest_points)

    # 2.b. Compute grid taking closest points extema on x and y
    max_x = np.max(closest_points[:, 0])
    min_x = np.min(closest_points[:, 0])
    num_x = int((max_x - min_x) / grid_step)
    max_y = np.max(closest_points[:, 1])
    min_y = np.min(closest_points[:, 1])
    num_y = int((max_y - min_y) / grid_step)
    x_grid = np.linspace(min_x, max_x, num_x)
    y_grid = np.linspace(min_y, max_y, num_y)
    X, Y = np.meshgrid(x_grid, y_grid)
    X, Y = X.ravel(), Y.ravel()
    pred_grids.append(np.column_stack([X, Y, np.zeros_like(X)]))

    # 3. Predict on the newly created grid
    Z_pred = model(X, Y)
    
    pred_points = np.column_stack([X, Y, Z_pred.ravel()])
    new_shell_points.append(pred_points)

new_shell_points = np.vstack(new_shell_points)
all_closest_points = np.vstack(all_closest_points)
pred_grids = np.vstack(pred_grids)

In [ ]:
viewer = napari.Viewer()
viewer.add_points(new_shell_points, size=0.1, face_color="green")
viewer.add_points(outer_shell.points, size=0.1)
viewer.add_points(all_closest_points, size=0.1, face_color="red")
viewer.add_points(pred_grids, size=0.1, face_color="blue")

<Points layer 'pred_grids' at 0x19cb4860f40>

##### 2. KNN interpolation
IDEA: Instead of placing grids arbitrarily on the gaps, do the following:
- Compute the distance from each point to its nearest neighbor (edge length)
- Now consider the points for which edge length exceeds a certain threshold (e.g. 2 * std). These points are likely to be placed on gaps.
- For each of such points, find the KNN and place a new point in the coordinate barycenter of the neighbors.

The result should be an evenly spaced grid!

CAZZATA -> If I consider the whole outer shell point cloud, I cannot find gaps in this way as the nearest neigbor for a given point will always be on the same mesh and not on a neighboring one. 

In [25]:
pc = o3d.geometry.PointCloud()
pc.points = o3d.utility.Vector3dVector(outer_shell.points)
nearest_distances = np.asarray(pc.compute_nearest_neighbor_distance())
mean_nearest_distances, std_nearest_distances = np.mean(nearest_distances), np.std(nearest_distances)

In [26]:
gap_mask = np.logical_or(
    nearest_distances > mean_nearest_distances + 2 * std_nearest_distances,
    nearest_distances < mean_nearest_distances - 2 * std_nearest_distances
)
gap_points = outer_shell.points[gap_mask]

In [27]:
viewer = napari.Viewer()
viewer.add_points(outer_shell.points, size=0.1)
viewer.add_points(gap_points, size=0.1, face_color="red")

<Points layer 'gap_points' at 0x19db2c3abc0>

In [ ]:
outer_shell.interpolate_gaps('spline')

##### 3. Linear Interpolation between mesh borders
ALGORITHM:
- Find "borders" of cell meshes (i.e., single layers of points that are the closest to another mesh). 
- For each mesh border:
    - For each point of the border:
        - Find closest point on the neighboring cell border
        - Sample points on the line that join these 2 points 

In [ ]:
# 1. Get all pairs of neighbors
neighbor_pairs = set()
for idx, neighbors in enumerate(outer_shell._neighbors_lst):
    for neighbor in neighbors:
        pair = tuple(sorted((idx, neighbor)))
        neighbor_pairs.add(pair)

# 2. Iterate over the neighbor pairs
for idx_1, idx_2 in neighbor_pairs:
    # 2.a. Get closest points for each cell in the pair
    mesh_1, mesh_2 = outer_shell._meshes[idx_1], outer_shell._meshes[idx_2]
    closest_point_idxs_1 = mesh_1.k_closest_dict[idx_2]
    closest_point_idxs_2 = mesh_2.k_closest_dict[idx_1]
    closest_points_1 = mesh_1.points[closest_point_idxs_1]
    closest_points_2 = mesh_2.points[closest_point_idxs_2]

    # 2.b.